In [1]:
!git clone https://github.com/tensorflow/examples
%cd examples/lite/examples/recommendation/ml/
!pip install -r requirements.txt

Cloning into 'examples'...
remote: Enumerating objects: 23729, done.
remote: Counting objects: 100% (693/693), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 23729 (delta 227), reused 591 (delta 166), pack-reused 23036
Receiving objects: 100% (23729/23729), 44.18 MiB | 26.24 MiB/s, done.
Resolving deltas: 100% (12876/12876), done.
/content/examples/lite/examples/recommendation/ml


Data Generation:

The script example_generation_movielens creates artificial examples of user interactions with movies. It considers factors like the minimum length of a user's history (timeline), the maximum context length (number of recent interactions considered), and the minimum rating a movie must have to be included. It divides the generated data into training and testing sets, and it also builds vocabularies to map movie IDs, genres, and ratings.

In [2]:
!python -m data.example_generation_movielens \
  --data_dir=data/raw \
  --output_dir=data/examples \
  --min_timeline_length=3 \
  --max_context_length=10 \
  --max_context_movie_genre_length=32 \
  --min_rating=2 \
  --train_data_fraction=0.9 \
  --build_vocabs=True

2024-01-09 07:42:54.753344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 07:42:54.753428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 07:42:54.755448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 07:42:54.764354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 07:42:56.083032: W tensorflow/compiler/tf2

In [3]:
!cat configs/sample_input_config.pbtxt

activity_feature_groups {
  features {
    feature_name: "context_movie_id"
    feature_type: INT
    vocab_size: 3953
    embedding_dim: 8
    feature_length: 10
  }
  features {
    feature_name: "context_movie_rating"
    feature_type: FLOAT
    feature_length: 10
  }
  encoder_type: CNN
}
activity_feature_groups {
  features {
    feature_name: "context_movie_genre"
    feature_type: STRING
    vocab_name: "movie_genre_vocab.txt"
    vocab_size: 19
    embedding_dim: 4
    feature_length: 32
  }
  encoder_type: CNN
}
label_feature {
  feature_name: "label_movie_id"
  feature_type: INT
  vocab_size: 3953
  embedding_dim: 8
  feature_length: 1
}


Model Training and Evaluation:

The recommendation_model_launcher script trains a recommendation model using TensorFlow. The model architecture is configured with hidden layers, convolutional layers (for feature extraction), and LSTM units (to capture temporal patterns). The training process involves adjusting the model's parameters based on the training data to improve its ability to predict movie preferences. The evaluation step assesses how well the model performs on a separate set of data not used during training (testing data). It measures the model's ability to recommend movies that users are likely to enjoy.

In [4]:
!python -m model.recommendation_model_launcher \
  --training_data_filepattern "data/examples/train_movielens_1m.tfrecord" \
  --testing_data_filepattern "data/examples/test_movielens_1m.tfrecord" \
  --model_dir "model/model_dir" \
  --export_dir "model/model_dir/export_m1" \
  --vocab_dir "data/examples" \
  --input_config_file "configs/sample_input_config.pbtxt" \
  --batch_size 32 \
  --learning_rate 0.01 \
  --steps_per_epoch 2 \
  --num_epochs 2 \
  --num_eval_steps 2 \
  --run_mode "train_and_eval" \
  --gradient_clip_norm 1.0 \
  --num_predictions 10 \
  --hidden_layer_dims "32,32" \
  --eval_top_k "1,5" \
  --conv_num_filter_ratios "2,4" \
  --conv_kernel_size 4 \
  --lstm_num_units 16

2024-01-09 07:46:53.712235: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 07:46:53.712316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 07:46:53.715305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 07:46:53.732404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 07:46:55.138176: W tensorflow/compiler/tf2

This command is executing the recommendation model launcher script with specific parameters to export a trained recommendation model. The script takes as input the file patterns for training and testing data in TensorFlow Record format, along with a configuration file defining model settings. The trained model checkpoints, stored in the directory specified by --model_dir during training, are exported to a new directory designated by --export_dir. Additionally, the script references a vocabulary directory (--vocab_dir) containing essential files for model understanding. The command runs in "export" mode, as indicated by --run_mode, specifying the checkpoint file (--checkpoint_path) from which to export the model. Various parameters, such as the number of predictions to generate, dimensions of hidden layers, evaluation metric thresholds, and convolutional layer configurations, are provided to tailor the export process. Once executed, the script saves the exported model in the specified directory, enabling its use for making predictions or deployment in other applications.

In [7]:
!python -m model.recommendation_model_launcher \
  --training_data_filepattern "data/examples/train_movielens_1m.tfrecord" \
  --testing_data_filepattern "data/examples/test_movielens_1m.tfrecord" \
  --input_config_file "configs/sample_input_config.pbtxt" \
  --model_dir "model/model_dir" \
  --export_dir "model/model_dir/export_m2" \
  --vocab_dir "data/examples" \
  --run_mode "export" \
  --checkpoint_path "model/model_dir/ckpt-4" \
  --num_predictions 10 \
  --hidden_layer_dims "32,32" \
  --eval_top_k "1,5" \
  --conv_num_filter_ratios "2,4" \
  --conv_kernel_size 4 \
  --lstm_num_units 16

2024-01-09 07:55:17.492469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 07:55:17.492547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 07:55:17.494015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 07:55:18.895532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:tensorflow:Setting up train and eval input datasets.
I0109 07:55:20.875624 132055901843456 app.py:254] Setting up train and eval input datasets.
INFO:tensorflow:Build keras model for mode: export.
I0109 07:55:21.257751 132055901843456 a

In [ ]:
import os
import tensorflow as tf

# Use [0, 1, ... 9] as example input to represent 10 movies that user interacted with.
context = tf.range(10)
# Path to exported TensorFlow Lite model.
tflite_model_path = 'model/model_dir/export/export_m2/model.tflite'  #@param {type:"string"}

# Create TFLite interpreter.
interpreter = tf.lite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Display inputs and outputs:')
print(input_details)
print(output_details)

# Find indices.
names = [
  'serving_default_context_movie_id:0',
  'serving_default_context_movie_genre:0',
  'serving_default_context_movie_rating:0',
]
indices = {i['name']: i['index'] for i in input_details}

# Fake inputs for illustration. Please change to the real data.
# Use [0, 1, ... 9] to represent 10 movies that user interacted with.
ids = tf.range(10)
interpreter.set_tensor(indices[names[0]], ids)
# Use [0, 1, ..., 31] to represent 32 movie genres.
genres = tf.range(32)
interpreter.set_tensor(indices[names[1]], genres)
# Use [1.0, 1.0, ..., 1.0] to represent 10 movie ratings.
ratings = tf.ones(10)
interpreter.set_tensor(indices[names[2]], ratings)

# Run inference.
interpreter.invoke()

# Get outputs.
top_prediction_ids = interpreter.get_tensor(output_details[0]['index'])
top_prediction_scores = interpreter.get_tensor(output_details[1]['index'])
print('Predicted results:')
print('Top ids: {}'.format(top_prediction_ids))
print('Top scores: {}'.format(top_prediction_scores))
